# 実際のWebページに対するスクレイピングのサンプル

以下は，いずれも動的Webページに対するサンプルですが，抽出部の指定については静的・動的に関わらず参考になるかもしれません．

## 準備

### ライブラリのインストール

In [1]:
!pip install requests-html lxml_html_clean

### htmlデータ取得用の関数定義

In [2]:
from requests_html import AsyncHTMLSession
from bs4 import BeautifulSoup

async def fetch_html(url):
    # AsyncHTMLSessionを作成
    session = AsyncHTMLSession()

    # ページを取得してレンダリング
    response = await session.get(url)
    await response.html.arender(timeout=30)

    return response

## Yahoo リアルタイム検索のトレンドの抽出

In [3]:
response = await fetch_html("https://search.yahoo.co.jp/realtime")

# レンダリング結果をBeautifulSoupで解析
soup = BeautifulSoup(response.html.html, 'html.parser')

# トレンド項目の抽出
trend_items = soup.select('.Trend_container__OK5FG h1')

# 抽出結果を表示
print("Yahoo!リアルタイム検索のトレンド項目:")
for i, item in enumerate(trend_items, start=1):
    print("{:2}. {}".format(i, item.text.strip()))



Yahoo!リアルタイム検索のトレンド項目:
 1. 田中将大
 2. 当選確実
 3. ジョッキーカメラ
 4. 球団発表
 5. 敗戦の弁
 6. 道長くん
 7. 減額制限
 8. 敦康親王
 9. 内閣総理大臣杯
10. 優勝先生
11. 広沢さん
12. サモエド
13. 行かないでくれ!
14. 優吾のあしあと
15. ミリしらパワプロ杯
16. 倫子さま
17. Dazzling Game
18. マーくん
19. 石破内閣支持率
20. 塩うに


## F1の2024 Driver Standingsの抽出

In [4]:
import pandas as pd

response = await fetch_html("https://www.formula1.com/en/results/2024/drivers")

# レンダリング結果をBeautifulSoupで解析
soup = BeautifulSoup(response.html.html, 'html.parser')

# 表の抽出
standings_table = soup.find('table', class_='f1-table')

# 表データの分割・DataFrameとして保存・表示
headers = [th.text.strip() for th in standings_table.find_all('th')]
rows = standings_table.find_all('tr')[1:]
data = []
for row in rows:
    cols = [td.text.strip() for td in row.find_all('td')]
    data.append(cols)
df = pd.DataFrame(data, columns=headers)
print(df)

   Pos               Driver Nationality                           Car  Pts
0    1    Max VerstappenVER         NED    Red Bull Racing Honda RBPT  403
1    2      Lando NorrisNOR         GBR              McLaren Mercedes  340
2    3   Charles LeclercLEC         MON                       Ferrari  319
3    4     Oscar PiastriPIA         AUS              McLaren Mercedes  268
4    5      Carlos SainzSAI         ESP                       Ferrari  259
5    6    George RussellRUS         GBR                      Mercedes  217
6    7    Lewis HamiltonHAM         GBR                      Mercedes  208
7    8      Sergio PerezPER         MEX    Red Bull Racing Honda RBPT  152
8    9   Fernando AlonsoALO         ESP  Aston Martin Aramco Mercedes   62
9   10   Nico HulkenbergHUL         GER                  Haas Ferrari   35
10  11      Yuki TsunodaTSU         JPN                 RB Honda RBPT   30
11  12      Pierre GaslyGAS         FRA                Alpine Renault   26
12  13      Lance StrollS